# Kolektor Surface Defect Detection
TMM4128 - Machine Learning for Engineers - Semestral Project


### Import TensorFlow

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import keras

In [ ]:
Categories = ["Undamaged", "Damaged"]
data_train = "./datasets/train/"
data_test = "./datasets/test/"
current_directory = os.getcwd()
print(current_directory)

flat_train = []
flat_train_GT = []
flat_test = []
flat_test_GT = []
damage_label_train = []
damage_label_test = []
img_name = []

In [ ]:
#Create path to folder
path = data_train

#For-loop to go through all the contents in the folder
for img in os.listdir(path):
    #Distinguishing between Groundtruth pictures and Actual pictures
    if "GT" in img:
        img_array = cv2.imread(os.path.join(path,img))
        flat_train_GT.append(img_array.flatten())
        #If the GT Image is Black, no damage is detected
        if np.all(img_array == 0):
            damage_label_train.append(False)
        else:
            damage_label_train.append(True)
        #Mark GT images, if any defect is on them
    elif "GT" not in img:
        img_array = cv2.imread(os.path.join(path,img))
        flat_train.append(img_array.flatten())

In [ ]:
#Create path to folder
path = os.path.join(current_directory, data_test)

#For-loop to go through all the contents in the folder
for img in os.listdir(path):
    #Distinguishing between Groundtruth pictures and Actual pictures
    if "GT" in img:
        img_array = cv2.imread(os.path.join(path,img))
        flat_test_GT.append(img_array.flatten())
        #If the GT Image is Black, no damage is detected
        if np.all(img_array == 0):
            damage_label_test.append(False)
        else:
            damage_label_test.append(True)
        #Mark GT images, if any defect is on them
    elif "GT" not in img:
        img_array = cv2.imread(os.path.join(path,img))
        flat_test.append(img_array.flatten())

In [ ]:
df_train = pd.DataFrame(flat_train)
df_train["Damage"] = damage_label_train

df_test = pd.DataFrame(flat_test)
df_test["Damage"] = damage_label_test

In [ ]:
X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1] 

X_test = df_test.iloc[:, -1] 
y_test = df_test.iloc[:, -1] 

In [ ]:
# Reshape
X_train = X_train.reshape(-1,231,636,1)
X_test = X_test.reshape(-1,231,636,1)
print("x_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
# Train - Test split

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2)
print("x_train shape",X_train.shape)
print("x_val shape",X_val.shape)
print("y_train shape",y_train.shape)
print("y_val shape",y_val.shape)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# Define the optimizer
# In neural networks, an optimizer is an algorithm used to adjust the weights and biases 
# of the network during training in order to minimize the loss function and improve the accuracy of the model.

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 100  
batch_size = 250

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=6)

In [ ]:
# Lets fit the model

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val, y_val), steps_per_epoch=X_train.shape[0] // batch_size,
                              callbacks=[early_stopping])

In [ ]:
print("Accuracy of the model is --> " , model.evaluate(X_val, y_val, batch_size=batch_size)[1]*100 , "%")
print("Loss of the model is --> " , model.evaluate(X_val, y_val, batch_size=batch_size)[0])

In [ ]:
# Model Loss

plt.figure()
plt.plot(history.history["loss"],label = "Train Loss", color = "black")
plt.plot(history.history["val_loss"],label = "Validation Loss", color = "darkred", marker = "+", linestyle="dashed",markeredgecolor = "purple", markeredgewidth = 2)
plt.title("Model Loss", color = "darkred", size = 13)
plt.legend()
plt.show()

In [ ]:
# Model Accuracy 

plt.figure()
plt.plot(history.history["accuracy"],label = "Train Accuracy", color = "black")
plt.plot(history.history["val_accuracy"],label = "Validation Accuracy", color = "darkred", marker = "+", linestyle="dashed",markeredgecolor = "purple", markeredgewidth = 2)
plt.title("Model Accuracy", color = "darkred", size = 13)
plt.legend()
plt.show()

In [ ]:
import matplotlib

# We make predictions using the model we have created.
Y_pred = model.predict(X_val)
# argmax = To briefly mention it, it will give the index of the value with the highest value.
Y_pred_classes = np.argmax(Y_pred,axis = 1) 

# We do the same for the y_val values. because we will compare these values. 
Y_true = np.argmax(y_val,axis = 1) 

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

# plot the confusion matrix
f,ax = plt.subplots(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label", color = "blue")
plt.ylabel("True Label", color = "green")
plt.title("Confusion Matrix", color = "darkred", size = 15)
plt.show()